In [1]:
import pandas as pd
import numpy as np
import platform as pf
import os
import xlrd

In [2]:
# delete duplicate file
def removeExist(path):
    if (os.path.exists(path)):
        os.remove(path)

In [3]:
# Before we start, please run these two commands to: 
# unzip raw_data_0422.zip to the root directory. You should see a data/ folder
# unzip raw_stock_history_0421.zip to the root directory. You should see a full_history/ folder

sys = pf.system()
if sys == 'Darwin' or sys == 'Linux':
    os.system('unzip -o ../raw_data_0422.zip -d ../')
    os.system('unzip -o ../raw_stock_history_0421.zip -d ../')
    
if sys == 'Windows':
    os.system('powershell -command "Expand-Archive" -Path "../raw_data_0422.zip" -DestinationPath "../" -Force')
    os.system('powershell -command "Expand-Archive" -Path "../raw_stock_history_0421.zip" -DestinationPath "../" -Force')

# If your opearting system does not support these commands, please unzip these two zip folders manually.

In [4]:
########################################
# Modify Covid-19 world data from JHU CSSE
def CleanCovid19Data():
    covidDataPath = '../data/covid-19/time_series_covid19_confirmed_global.csv'
    covidDataNewPath = '../data/covid-19/time_series_covid19_confirmed_global_modified.csv'
    df = pd.read_csv(covidDataPath)
    # remove useless columns
    df = df.drop(['Province/State', 'Lat', 'Long'], 1, errors='ignore')
    if 'Country/Region' in df.columns.values:
        df.rename(columns={'Country/Region': 'Date'}, inplace=True)
        # transform date format to 'yyyy-mm-dd'
        # df.rename(lambda x: modifyDate(x), axis = 'columns', inplace = True)
        # combine province/state data to the whole country
        group_df = df[1:].groupby(df['Date'])
        sum_df = group_df.sum()
        # calculate world data
        sum_df.loc["World"] = sum_df.apply(lambda x: x.sum())
        # transposition, probably not necessary
        result = pd.DataFrame(sum_df.values.T, index=sum_df.columns, columns=sum_df.index)
        # transform date format to 'yyyy-mm-dd'
        result.index = pd.to_datetime(result.index)
        result.index.name = 'Date'
        # output
    result.to_csv(covidDataNewPath, index=True, header=True)

In [5]:
# Sample: Initial structure of COVID data
COVID_path = '../data/covid-19/time_series_covid19_confirmed_global.csv'
COVID = pd.read_csv(COVID_path)
COVID.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,665,714,784,840,906,933,996,1026,1092,1176
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,467,475,494,518,539,548,562,584,609,634
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,1983,2070,2160,2268,2418,2534,2629,2718,2811,2910
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,646,659,673,673,696,704,713,717,717,723
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,19,19,19,19,19,24,24,24,24,25


In [6]:
# Execute CleanCovid19Data
CleanCovid19Data()

In [7]:
# Now we get modified COVID 19 data
COVID_path = '../data/covid-19/time_series_covid19_confirmed_global_modified.csv'
COVID = pd.read_csv(COVID_path)
COVID.head()

,Date,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe,World
0,2020-01-22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,555
1,2020-01-23,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,654
2,2020-01-24,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,941
3,2020-01-25,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,1434
4,2020-01-26,0,0,0,0,0,0,0,4,0,...,0,0,0,2,0,0,0,0,0,2118


In [8]:
########################################
# Transform every date to format "yyyy-mm-dd"
def DateTransformation():
    '''
    this function is quite dump, it walk through the whole data directory, and try to modify every csv file
    but it some how make sense since most of our file need to be transformed
    THIS WILL WORK ONLY WHEN DATE DATA NEED TO BE TRANSFORMED HAVE COLUMN NAME 'Date' 
    '''
    rootPath = "../data"
    for root, dirs, files in os.walk(rootPath):
        for file in files:
            path = os.path.join(root, file)
            if path.endswith('.csv'):
                df = pd.read_csv(path)
                if 'DATE' in df.columns:
                    df.rename(columns={'DATE': 'Date'}, inplace=True)
                if 'date' in df.columns:
                    df.rename(columns={'date': 'Date'}, inplace=True)
                if 'Date' in df.columns:
                    df['Date'] = pd.to_datetime(df['Date'])
                df.to_csv(path, index=False, header=True)

In [9]:
# Sample: Initial structure of Brent, the date format is mm/dd/yyyy
Brent_path = '../data/market/Commodities/Energies/BrentCrude.csv'
Brent = pd.read_csv(Brent_path)
Brent.head()

,Date,Close/Last,Volume,Open,High,Low
0,04/21/2020,19.33,82076,19.13,20.22,18.32
1,04/20/2020,25.57,44255,25.47,26.54,25.38
2,04/17/2020,28.08,34615,28.47,29.00,27.60
3,04/16/2020,27.82,29580,28.06,29.01,27.20
4,04/15/2020,27.69,29940,30.01,30.42,27.17


In [10]:
# Execute DateTransformation
DateTransformation()

In [11]:
Brent = pd.read_csv(Brent_path)
Brent.head()

,Date,Close/Last,Volume,Open,High,Low
0,2020-04-21,19.33,82076,19.13,20.22,18.32
1,2020-04-20,25.57,44255,25.47,26.54,25.38
2,2020-04-17,28.08,34615,28.47,29.00,27.60
3,2020-04-16,27.82,29580,28.06,29.01,27.20
4,2020-04-15,27.69,29940,30.01,30.42,27.17


In [12]:
########################################
# Trans currencies to USD base
def TransToUSDBase():
    currenciesPath = '../data/market/Currencies/'
    currencies = ['AUD_USD.csv', 'EUR_USD.csv', 'GBP_USD.csv']
    for currency in currencies:
        curPath = currenciesPath + currency
        newPath = currenciesPath + currency[4:7] + '_' + currency[:3] + '.csv'
        df = pd.read_csv(curPath)
        columns = [df.columns.values[1], df.columns.values[3], df.columns.values[4], df.columns.values[5]]
        for col in columns:
            df[col] = df[col].apply(lambda x: 1 / x)
        removeExist(newPath)
        df.to_csv(newPath, index=False, header=True)

In [13]:
# Sample: Initial structure of EUR_USD.csv
EUR_USD_path = '../data/market/Currencies/EUR_USD.csv'
EUR_USD = pd.read_csv(EUR_USD_path)
EUR_USD.head()

,Date,Close/Last,Volume,Open,High,Low
0,2020-04-20,1.0840,N/A,1.0862,1.0872,1.0825
1,2020-04-19,1.0859,N/A,1.0876,1.0878,1.0857
2,2020-04-17,1.0872,N/A,1.0835,1.0892,1.0813
3,2020-04-16,1.0869,N/A,1.0835,1.0880,1.0835
4,2020-04-15,1.0873,N/A,1.0907,1.0913,1.0873


In [14]:
# Execute DealWithCurrencies()
TransToUSDBase()

In [15]:
# Now we have USD to EUR currencies data
USD_EUR_path = '../data/market/Currencies/USD_EUR.csv'
USD_EUR = pd.read_csv(USD_EUR_path)
USD_EUR.head()

,Date,Close/Last,Volume,Open,High,Low
0,2020-04-20,0.922509,N/A,0.920641,0.919794,0.923788
1,2020-04-19,0.920895,N/A,0.919456,0.919287,0.921065
2,2020-04-17,0.919794,N/A,0.922935,0.918105,0.924813
3,2020-04-16,0.920048,N/A,0.922935,0.919118,0.922935
4,2020-04-15,0.919709,N/A,0.916842,0.916338,0.919709


In [16]:
########################################
# Trans Yahoo Data Format to NASDAQ Data Format
def TransYahooToNASDAQ():
    path_index = '../data/market/Index/'
    file_path = []
    file_path.append(path_index + 'CBOE_VolatilityIndex.csv')
    file_path.append(path_index + 'DowJones_IndustrialAvg.csv')
    file_path.append(path_index + 'NASDAQ_100_Index.csv')
    file_path.append(path_index + 'Russell_2000.csv')
    for path in file_path:
        df = pd.read_csv(path)
        # remove redundant column 'Adj Close'
        df = df.drop('Adj Close', axis=1, errors='ignore')
        # rename Close to Close/Last and put to second place
        if 'Close' in df.columns.values:
            df_close = df.Close
            df = df.drop('Close', axis=1, errors='ignore')
            df.insert(1, 'Close/Last', df_close)
        # reorder in time
        df['Date'] = pd.to_datetime(df['Date'])
        df.sort_values('Date', inplace=True, ascending=False)
        # save
        df.to_csv(path, index=False, header=True)

In [17]:
# Sample: Initial structure of DowJones_IndustrialAvg in Yahoo format
DJI_yahoo_path = '../data/market/Index/DowJones_IndustrialAvg.csv'
DJI_yahoo = pd.read_csv(DJI_yahoo_path)
DJI_yahoo.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-04-21,18034.230469,18109.699219,17929.630859,17949.589844,17949.589844,95180000
1,2015-04-22,17950.820313,18056.019531,17887.759766,18038.269531,18038.269531,91260000
2,2015-04-23,18031.900391,18133.029297,17966.769531,18058.689453,18058.689453,100240000
3,2015-04-24,18056.419922,18108.869141,18009.080078,18080.140625,18080.140625,119130000
4,2015-04-27,18097.890625,18175.560547,18024.660156,18037.970703,18037.970703,121110000


In [18]:
# Execute TransYahooToNASDAQ()
TransYahooToNASDAQ()

In [19]:
# Now the structure of DowJones_IndustrialAvg is in NASDAQ format
DJI_yahoo = pd.read_csv(DJI_yahoo_path)
DJI_yahoo.head()

,Date,Close/Last,Open,High,Low,Volume
0,2020-04-20,23650.439453,24095.099609,24108.689453,23627.189453,423410000
1,2020-04-17,24242.490234,23817.150391,24264.210938,23817.150391,525950000
2,2020-04-16,23537.679688,23543.660156,23598.080078,23211.380859,471050000
3,2020-04-15,23504.349609,23600.720703,23649.720703,23233.320313,437630000
4,2020-04-14,23949.759766,23690.570313,24040.580078,23683.439453,485910000


In [20]:
########################################
# Clean Data From NASDAQ (Excepet All Stock Data)
def CleanDataFromNASDAQ():
    file_idx = 'NasdaqSource.txt'
    file_path = {}
    for line in open(file_idx):
        path = line.strip('\n')
        name = path.split("/")[-1][:-4]
        file_path[name] = path

    for name in file_path:
        print(name + ': ' + file_path[name])

    for path in file_path.values():
        df = pd.read_csv(path)
        # format column name, delete redundant whitespace in column names
        df.columns = [x.strip() for x in df.columns.values if x.strip() != '']
        # remove redundant column 'Volume'
        df = df.drop('Volume', axis=1, errors='ignore')
        # save
        df.to_csv(path, index=False, header=True)
        # print(path)
        # print(df.columns)

In [21]:
# Sample: Initial structure of NASDAQ_Composite
NASDAQ_comp_path = '../data/market/Index/NASDAQ_Composite.csv'
NASDAQ_comp = pd.read_csv(NASDAQ_comp_path)
NASDAQ_comp.head()

,Date,Close/Last,Volume,Open,High,Low
0,2020-04-21,8263.23,0,8460.69,8480.29,8215.69
1,2020-04-20,8560.73,0,8553.37,8684.91,8553.37
2,2020-04-17,8650.14,0,8667.48,8670.30,8531.69
3,2020-04-16,8532.36,0,8479.11,8560.16,8393.27
4,2020-04-15,8393.18,0,8355.96,8464.66,8308.79


In [22]:
# Execute CleanDataFromNASDAQ() and print all data collected from NASDAQ which will be modified
CleanDataFromNASDAQ()

BrentCrude: ../data/market/Commodities/Energies/BrentCrude.csv
CrudeOil_CL: ../data/market/Commodities/Energies/CrudeOil_CL.csv
Ethanol: ../data/market/Commodities/Energies/Ethanol.csv
Gasoline: ../data/market/Commodities/Energies/Gasoline.csv
HeatingOil: ../data/market/Commodities/Energies/HeatingOil.csv
NaturalGas: ../data/market/Commodities/Energies/NaturalGas.csv
Corn: ../data/market/Commodities/Grains/Corn.csv
Oat: ../data/market/Commodities/Grains/Oat.csv
RedWheat: ../data/market/Commodities/Grains/RedWheat.csv
RoughRice: ../data/market/Commodities/Grains/RoughRice.csv
SoybeanMeal: ../data/market/Commodities/Grains/SoybeanMeal.csv
SoybeanOil: ../data/market/Commodities/Grains/SoybeanOil.csv
Soybeans: ../data/market/Commodities/Grains/Soybeans.csv
Wheat: ../data/market/Commodities/Grains/Wheat.csv
FeederCattle: ../data/market/Commodities/Meats/FeederCattle.csv
LeanHogs: ../data/market/Commodities/Meats/LeanHogs.csv
LiveCattle: ../data/market/Commodities/Meats/LiveCattle.csv
Milk: 

In [23]:
# Now the redundant columns and blank space in column names are deleted
NASDAQ_comp = pd.read_csv(NASDAQ_comp_path)
NASDAQ_comp.head()

,Date,Close/Last,Open,High,Low
0,2020-04-21,8263.23,8460.69,8480.29,8215.69
1,2020-04-20,8560.73,8553.37,8684.91,8553.37
2,2020-04-17,8650.14,8667.48,8670.30,8531.69
3,2020-04-16,8532.36,8479.11,8560.16,8393.27
4,2020-04-15,8393.18,8355.96,8464.66,8308.79


In [24]:
########################################
# Clean Data From Other Source
def CleanOtherSources():
    file_CrudeOil_WTI_MacroTrends = '../data/market/Commodities/Energies/CrudeOil_WTI_macrotrends.csv'
    file_Finance_Sector_Related_Policy_Responses = '../data/general/FinanceSectorRelatedPolicyResponses.xlsx'
    file_Industrial_Production_Index = '../data/general/IndustrialProductionIndex.csv'
    file_Initial_Jobless_Claims = '../data/employment/InitialJoblessClaims.csv'
    file_Unemployment_Rate = '../data/employment/UnemploymentRate.csv'

    # Finance Sector Related Policy Responses
    FSRPR = pd.read_excel(file_Finance_Sector_Related_Policy_Responses, sheet_name='Raw data')
    # drop redundant columns
    FSRPR = FSRPR.drop('Iso 3 Code', axis=1, errors='ignore')
    FSRPR = FSRPR.drop('Entry date', axis=1, errors='ignore')
    # reorder
    FSRPR_Country = FSRPR.Country
    FSRPR = FSRPR.drop('Country', axis=1, errors='ignore')
    FSRPR.insert(0, 'Country', FSRPR_Country)
    FSRPR.to_csv(file_Finance_Sector_Related_Policy_Responses[:-4] + 'csv', index=False, header=True)

    # CrudeOil WTI MacroTrends
    CWM = pd.read_csv(file_CrudeOil_WTI_MacroTrends)
    # reorder in time
    CWM['Date'] = pd.to_datetime(CWM['Date'])
    CWM.sort_values('Date', inplace=True, ascending=False)
    # format column name, delete redundant whitespace in column names
    CWM.columns = [x.strip() for x in CWM.columns.values if x.strip() != '']
    # delete invalid future data
    CWM.dropna(subset=['value'], inplace=True)
    CWM.to_csv(file_CrudeOil_WTI_MacroTrends, index=False, header=True)
    
    
    # Others with date (reorder date)
    file_path = []
    file_path.append(file_Industrial_Production_Index)
    file_path.append(file_Initial_Jobless_Claims)
    file_path.append(file_Unemployment_Rate)
    for path in file_path:
        df = pd.read_csv(path)
        # reorder in time
        df['Date'] = pd.to_datetime(df['Date'])
        df.sort_values('Date', inplace=True, ascending=False)
        # save
        df.to_csv(path, index=False, header=True)

In [25]:
# Sample: Initial structure of Finance Sector Related Policy Responses
FSRPR_path = '../data/general/FinanceSectorRelatedPolicyResponses.xlsx'
FSRPR = pd.read_excel(FSRPR_path, sheet_name='Raw data')
FSRPR.head()

,Iso 3 Code,Level of income,Region,Country,Date (at or prior to),Entry date,Level 1 policy measure,Level 2 policy measure
0,CHN,Upper middle income,EAP,China,2020-02-01,2020-03-21,Financial Institutions,Operational continuity
1,CAN,High income,Other G20,Canada,2020-02-03,2020-03-21,Financial Markets,Market functioning
2,THA,Upper middle income,EAP,Thailand,2020-02-05,2020-03-21,Liquidity/funding,Policy rate
3,RUS,Upper middle income,ECA,Russian Federation,2020-02-07,2020-03-21,Liquidity/funding,Policy rate
4,BLR,Upper middle income,ECA,Belarus,2020-02-19,2020-03-31,Liquidity/funding,Liquidity (incl FX)/ELA


In [26]:
# Execute CleanOtherSources()
CleanOtherSources()

In [27]:
# Now the file is in csv format and redundant columns are deleted
FSRPR = pd.read_csv('../data/general/FinanceSectorRelatedPolicyResponses.csv')
FSRPR.head()

,Country,Level of income,Region,Date (at or prior to),Level 1 policy measure,Level 2 policy measure
0,China,Upper middle income,EAP,2020-02-01,Financial Institutions,Operational continuity
1,Canada,High income,Other G20,2020-02-03,Financial Markets,Market functioning
2,Thailand,Upper middle income,EAP,2020-02-05,Liquidity/funding,Policy rate
3,Russian Federation,Upper middle income,ECA,2020-02-07,Liquidity/funding,Policy rate
4,Belarus,Upper middle income,ECA,2020-02-19,Liquidity/funding,Liquidity (incl FX)/ELA


In [28]:
########################################
# Clean Data From NASDAQ (All stock data and company information)

def CleanAllHistory():
    all_symbol = []
    # check two stock source files and use their intersection
    fp = open('../data/market/Stock/all symbols/all_symbols.txt')
    for line in fp:
        all_symbol.append(line.strip())
    fp = open('../data/market/Stock/all symbols/excluded_symbols.txt')
    for line in fp:
        all_symbol.append(line.strip()) 
    fp.close()
    
    company_list_path = '../data/market/Stock/company info/companylist.csv'
    company_list = pd.read_csv(company_list_path)
    # drop some redundant columns
    company_list = company_list.drop('ADR TSO', axis=1, errors='ignore')
    company_list = company_list.drop('LastSale', axis=1, errors='ignore')
    company_list = company_list.drop('IPOyear', axis=1, errors='ignore')
    company_list = company_list.drop('Summary Quote', axis=1, errors='ignore')
    # must ensure that sector is not NaN
    company_list = company_list.dropna(subset=['Sector'])
    
    company_list.to_csv('../data/market/Stock/company info/NewCompanyList.csv', index=False, header=True)
    company_list = company_list['Symbol'].values.tolist()
    
    final_list = list(set(all_symbol).intersection(set(company_list)))
    
    # generate a new directory to store all modified stock history
    old_path = '../full_history/'
    new_path = '../data/market/Stock/history/'
    if not os.path.exists(new_path):
        os.mkdir(new_path)
    
    counter = 0
    for each_file in final_list:
        counter += 1
        if counter % 500 == 0:
            print('Progress: ' + str(counter) + '/' + str(len(final_list)))
        
        file_path = old_path + each_file + '.csv'
        try:
            file = pd.read_csv(file_path)
        except:
            continue
        
        # drop close column
        file = file.drop('close', axis=1, errors='ignore')
        
        # filter and use recent 2 years' record
        file = file[file['date'] >= '2018-01-01']
        
        file.to_csv(new_path + each_file + '.csv', index=False, header=True)     

In [29]:
# Sample: Initial structure of company list
company_list_path = '../data/market/Stock/company info/companylist.csv'
company_list = pd.read_csv(company_list_path)
company_list.head()

,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote
0,A,"Agilent Technologies, Inc.",77.37,2.395773e+10,NaN,1999.0,Capital Goods,Biotechnology: Laboratory Analytical Instruments,https://old.nasdaq.com/symbol/a
1,AA,Alcoa Corporation,7.45,1.385069e+09,NaN,2016.0,Basic Industries,Aluminum,https://old.nasdaq.com/symbol/aa
2,AACG,ATA Creativity Global,0.83,9.686664e+06,11670680.0,NaN,Consumer Services,Other Consumer Services,https://old.nasdaq.com/symbol/aacg
3,AAL,"American Airlines Group, Inc.",11.06,4.712210e+09,NaN,NaN,Transportation,Air Freight/Delivery Services,https://old.nasdaq.com/symbol/aal
4,AAMC,Altisource Asset Management Corp,13.92,2.253370e+07,NaN,NaN,Finance,Real Estate,https://old.nasdaq.com/symbol/aamc


In [30]:
CleanAllHistory()

Progress: 500/5279
Progress: 1000/5279
Progress: 1500/5279
Progress: 2000/5279
Progress: 2500/5279
Progress: 3000/5279
Progress: 3500/5279
Progress: 4000/5279
Progress: 4500/5279
Progress: 5000/5279


In [31]:
# Now the redundant columns are deleted
company_list_path = '../data/market/Stock/company info/NewCompanyList.csv'
company_list = pd.read_csv(company_list_path)
company_list.head()

,Symbol,Name,MarketCap,Sector,Industry
0,A,"Agilent Technologies, Inc.",2.395773e+10,Capital Goods,Biotechnology: Laboratory Analytical Instruments
1,AA,Alcoa Corporation,1.385069e+09,Basic Industries,Aluminum
2,AACG,ATA Creativity Global,9.686664e+06,Consumer Services,Other Consumer Services
3,AAL,"American Airlines Group, Inc.",4.712210e+09,Transportation,Air Freight/Delivery Services
4,AAMC,Altisource Asset Management Corp,2.253370e+07,Finance,Real Estate
